# Project 4 - Circuit 4C: DIY Who Am I? Game

"DIY Who Am I?" is based on the popular [Hedbanz](http://www.hedbanz.com/) game or [HeadsUp!](https://www.warnerbros.com/videogame/heads) app. It's a fun party game in which a player holds an LCD screen to his/her forehead so that the player can’t see the word(s) that appear on the screen. Other players have to give hints, act out charades or make noises that will make the player with the LCD guess the word(s).
 
TODO: Update the LCD display with an OLED display.
TODO: Update the jumper wires with a Qwiic connector.
TODO: Remove potentiometer
![What you need](images/sik-demo-prj4-cc-need.jpg)

## New Components

### 4xAA Battery Holder
Included in your kit is a 4-cell AA battery holder. The 5-inch cable is terminated with a standard barrel jack connector. The connector mates with the barrel jack on the RedBoard, allowing you to easily make your project battery powered.

![Battery Pack](images/sik-docs-prj4-cb-battery-pack.jpg)

## New Concepts

### Button Debounce
When working with momentary buttons, it is usually necessary to add button debouncing to your code. This is because the code that is meant to execute when the button is pressed may execute faster than you can press and release the button (microcontrollers are fast!). The simplest way to debounce a button is to add a small delay to the end of your code. This script adds a 500 millisecond delay at the end of the infinite loop to account for this. This simple addition will prevent a word from getting skipped when you press the button for the game.

### "Magic" Save Command
Jupyter notebooks do not have a built-in way to "save" or "flash" code to a device, meaning put code on a device that will run automatically whenever the RedBoard is restarted. That is why we have added a "Magic" command for saving code to your RedBoard. This is a command that isn't a built-in part of Python or Jupyter notebooks, but is specific to SparkFun Jupyter Notebooks. To save all the code from a cell to a device, put this exact line in your code:

```python 
# %save%
```

If you are connected to your device, this command should save all of the other code that is below it in a cell. This will replace all the code that is currently saved to the device, so to save an entire program, we must put it all in the same cell under the `# %save%` line.

## Lists of Strings
In circuit 2A you used a list of characters to represent musical notes. In this program, you’ll want to make a list of strings. Strings use multiple characters to make words.

```python
listOfStrings = [“Feynman” “Sagan”, “Tyson”, “Nye”]
```

## Hardware Hookup

Batteries are polarized. They have a positive end and a negative end. The battery holder has images indicating which end goes in which orientation for each cell.

![BatteryPolarity](images/sik-docs-prj4-cc-battery-pol.png)

Ensure all the batteries are inserted correctly before plugging the battery holder into the RedBoard.

### Battery Holder Attachment
To attach the battery holder to the breadboard baseplate, first cut two strips of Dual Lock that are roughly 1 inch x 1 inch each, or 2.5cm x 2.5cm. Remove the adhesive backing and attach one piece to the back of the battery holder.

![Battery Dual Lock](images/sik-docs-prj4-cc-battery-dl.jpg)

Adhere the second piece to the bottom of the breadboard baseplate (directly in the middle is recommended, as this will come into play in Project 5).

![Baseplate Dual Lock](images/sik-docs-prj4-cc-bp-dl.jpg)

Last, press the battery holder to the baseplate so that the two pieces of Dual Lock snap together. Insert the batteries into the holder if you have not done so already. Remember that batteries are polarized and can only go in one way.

![Battery Insert](images/sik-docs-prj4-cc-battery-ins.jpg)

![Battery Complete](images/sik-docs-prj4-cc-battery-complete.jpg)

Remove the battery pack while building your circuit.

TODO: Should we even have the below note?
NOTE: If you would like, feel free to get creative with some of the dual lock tape, the binder clip, screws, or any other method you see fit to attach the OLED to your baseplate. This is not mandatory, but is somewhat nice for holding the game up to your head while playing. NOTE: DO NOT WASTE ANYMORE THAN A 1"x1" SECTION OF DUAL LOCK TO ATTACH THE OLED. MOST OF THE DUAL LOCK IS USED FOR THE ROBOT IN PROJECT 5.

Ready to start hooking everything up? Check out the circuit diagram and hookup table below to see how everything is connected.

### Circuit Diagram

TODO: Replace with new fritzing showing the qwiic conection between the OLED and the RedBoard RP2350
![Hookup](images/sik-docs-prj4-cc-hookup.jpg)

**Note for Advanced Users**: If you know how to read datasheets and schematics, you can also refer to the schematic below as an alternative.

TODO: Replace with new schematic showing the qwiic conection between the OLED and the RedBoard RP2350
![Schematic](images/sik-docs-prj4-cc-schem.jpg)

### Hookup Table

TODO: Change to match the new simple connection of:
 5V -> temp +V 
 GND -> temp GND
 A0 -> temp SIGNAL
 OLED Qwiic -> RedBoard Qwiic

| Component        | RedBoard         | Breadboard     | Breadboard     | Breadboard     |
|------------------|------------------|----------------|----------------|----------------|
| Jumper Wire      | 5V               | 5V Rail ( + )  |                |                |
| Jumper Wire      | GND              | GND Rail ( - ) |                |                |
| LCD              |                  | A15-A30 (Pin 1 on A15) |        |                |
| Jumper Wire      |                  | E30            | GND Rail ( - ) |                |
| Jumper Wire      |                  | E29            | 5V Rail ( + )  |                |
| Jumper Wire      | Digital Pin 8    | E28            |                |                |
| Jumper Wire      | Digital Pin 9    | E27            |                |                |
| Jumper Wire      | Digital Pin 10   | E26            |                |                |
| Jumper Wire      | Digital Pin 11   | E25            |                |                |
| Jumper Wire      | Digital Pin 12   | E20            |                |                |
| Jumper Wire      |                  | E19            | GND Rail ( - ) |                |
| Jumper Wire      | Digital Pin 13   | E18            |                |                |
| Jumper Wire      |                  | E16            | 5V Rail ( + )  |                |
| Jumper Wire      |                  | E15            | GND Rail ( - ) |                |
| Potentiometer    |                  | A8             | A9             | A10            |
| Jumper Wire      |                  | E9             | E17            |                |
| Jumper Wire      |                  | E8             | GND Rail ( - ) |                |
| Jumper Wire      |                  | E10            | 5V Rail ( + )  |                |
| Buzzer           |                  | G6             |                |                |
| (Buzzer +)       |                  | G8             |                |                |
| (Buzzer -)       |                  |                |                |                |
| Jumper Wire      | Digital Pin 6    | J6             |                |                |
| Jumper Wire      |                  | J8             | GND Rail ( - ) |                |
| Push Button      |                  | D1/D3          | G1/G3          |                |
| Jumper Wire      | Digital Pin 2    | J1             |                |                |
| Jumper Wire      |                  | J3             | GND Rail ( - ) |                |


## Using the OLED.

Now that your circuit is built, it's time to write to the OLED. This is done using MicroPython, which is running on the RedBoard.

The first step is to connect your RedBoard to a USB port on this computer.

Select the "Connect" button at the bottom right of this screen and a panel is displayed

Select the "Connect Device" Button, and when the selection dialog appears, select the port with that displays ***Board in FS mode (...)*** or *TBD*

![Select a Port](images/sik-demo-select-port.png)

With the RedBoard connected, use the following MicroPython commands to write to the OLED. 

### Using MicroPython

The following MicroPython commands are entered to write to the OLED. 

**REMEMBER** To enter a MicroPython command, hold down either the Control (on Windows) or Command (on Mac) key when pressing *Enter*

#### Saving Code to Your Board

This time we're going to do something a bit different, we are going to have all of our code in a single Jupyter Notebook cell so that it all gets saved to our board at once when we run it. We'll put together our buzzer functions from Project 2 the Qwiic OLED module we used in circuits 4A and 4B, along with some button control to make our own guessing game. Run the cell below to save this code to your board and then unplug it, plug in the battery pack and watch it go! 

NOTE: If you'd rather run the game powered from a chord and in the Jupyter Notebook, you can remove the save line in the code below, not attach the battery pack, and run the code in the Jupyter Notebook like normal instead. 

In [ ]:
# The below "save" comment is a SparkFun Jupyter Notebook command to save all code below to the board permanently
# %save%

# The qwiic_oled driver module allows for control of SparkFun OLEDs.
# The QwiicLargeOled class is for the Large 1.3" OLED display in our SIK
from qwiic_oled import QwiicLargeOled
from machine import PWM # Allows us to use "PWM" (pulse-width modulation) to control the brightness of our LED
from machine import Pin # Allows us to use "Pin" to use code to interface with the pins on our board
from time import sleep_ms # Import the sleep_ms function to pause execution for a specified number of milliseconds
from random import randint # Import the randint function to generate random integers
from time import ticks_ms # function to get the current time in milliseconds

# Define the OLED object that we will use
# Note how we don't have to provide the pins, the driver automatically selects the pins for the qwiic connector
myOLED = QwiicLargeOled()

# Create a PWM object on pin 34 with a frequency of 0Hz and an initial "on time" of 0 (off)
pwmSpeaker  = PWM(Pin(34), freq=0, duty_u16=0)

# Create a Pin object for the button on pin 33, set as input with pull-up resistor
buttonPin = Pin(33, Pin.IN, Pin.PULL_UP) 

# Function for playing a timed tone from the speaker (buzzer)
def timed_tone(frequency, duration):
    pwmSpeaker.freq(frequency) # Set the frequency of the speaker
    pwmSpeaker.duty_u16(32768) # Set the duty cycle to 50%
    sleep_ms(duration) # Wait for the specified duration
    pwmSpeaker.duty_u16(0) # Turn off the speaker by setting duty cycle to 0

words = [ "moose", "beaver", "bear", "goose", "dog", "cat", "squirrel", "bird", "elephant", "horse",
          "bull", "giraffe", "seal", "bat", "skunk", "turtle", "whale", "rhino", "lion", "monkey",
          "frog", "alligator", "kangaroo", "hippo", "rabbit" ]

# Create a list of 25 elements to store the order to display the words
# This will store indices into the words list, which will be used to display the words in a random order.
# We will initialize the list with -1 (which is an invalid index) until we have 
# created the random order of words with our generate_random_order() function
# If you change the number of words in the words list, you will need to change the size of this list as well.
orderOfWords = [-1, -1, -1, -1, -1, -1, -1, -1,
                -1, -1, -1, -1, -1, -1, -1, -1,
                -1, -1, -1, -1, -1, -1, -1, -1, -1]

# Pro Tip: The above list could also be created in a more compact way by...
#
# using "list comprehension":
# orderOfWords = [-1 for _ in range(len(words))] 
#
# OR using "list initialization":
# orderOfWords = [-1] * len(words)

timeLimit = 15000 # Time limit for each word in milliseconds (15 seconds)
roundNumber = 0 # Keeps track of the current round number so it can be displayed on the OLED

# Function to generate a random order of words
def generate_random_order():
    for i in range(len(orderOfWords)):
        # Generate a random index from 0 to the length of the words list
        random_index = randint(0, len(words) - 1)
        
        # Keep generating a new random index until we find one that is not already in the orderOfWords list
        # This ensures that each word will be used exactly once in the order
        while random_index in orderOfWords:
            random_index = randint(0, len(words) - 1)
        
        # Assign the random index to the orderOfWords list
        orderOfWords[i] = random_index

# Function to fully clear the OLED display
def clear_all_oled():
    myOLED.clear(myOLED.ALL) # Clear OLED graphic memory.
    myOLED.clear(myOLED.PAGE) # Clear the processor's display buffer.

def show_start_sequence():
    myOLED.set_font_type(1) # Set our font to font #1 (medium sized letters and numbers)
    clear_all_oled()  # Clear the OLED display
    myOLED.set_cursor(0, 0)  # Move the cursor to the top left corner
    myOLED.print("Category:")  # Print "Category:"
    myOLED.display() # Actually show what we've printed
    
    myOLED.set_cursor(0, 50)  # Move the cursor to the bottom left corner
    myOLED.print("Animals")  # Print "Animals"
    myOLED.display() # Actually show what we've printed
    
    sleep_ms(2000)  # Wait for 2 seconds
    
    clear_all_oled()  # Clear the display
    myOLED.set_cursor(20, 25)  # Move the cursor to middle(ish) of the screen
    myOLED.print("Get ready!")  # Print "Get ready!"
    myOLED.display() # Actually show what we've printed
    sleep_ms(1000)  # Wait for 1 second

    # Let's make our countdown nice and big by using font #3 (Large Number Font)
    myOLED.set_font_type(3)
    
    # Pro Tip: You can count DOWN with a for loop by passing -1 to the range function
    # This will count from 3 to 1, decreasing by 1 each time
    for i in range(3, 0, -1):  # Countdown from 3 to 1
        clear_all_oled()  # Clear the display
        myOLED.set_cursor(60, 10)  # Move the cursor to middle(ish) of the screen
        myOLED.print(str(i))  # Print the countdown number
        myOLED.display() # Actually show what we've printed
        sleep_ms(1000)  # Wait for 1 second

    # # Set our font back to font #1 (medium sized letters and numbers) after the countdown
    myOLED.set_font_type(1)

# Function to display when the player loses
def game_over():
    clear_all_oled()  # Clear the OLED display
    myOLED.set_cursor(0, 0)  # Move the cursor to the top left corner
    myOLED.print("Game Over")  # Print "Game Over"
    
    myOLED.set_cursor(0, 50)  # Move to the bottom row
    myOLED.print("Score: ")  # Print a label for the score
    myOLED.print(roundNumber - 1)  # Print the score (the score is equal to the previous level/round number)
    myOLED.display() # Actually show what we've printed
    
    # Play the losing fog horn
    timed_tone(130, 250)  # E6
    sleep_ms(275)
    timed_tone(73, 250)   # G6
    sleep_ms(275)
    timed_tone(65, 150)   # E7
    sleep_ms(175)
    timed_tone(98, 500)   # C7
    sleep_ms(500)

# Function to display when the player wins
def winner():
    clear_all_oled()  # Clear the OLED display
    myOLED.set_cursor(50, 20)  # Move the cursor to the top center of the screen
    myOLED.print("YOU")  # Print "You"
    
    myOLED.set_cursor(50, 40)  # Move to the bottom center of the screen
    myOLED.print("WIN!")  # Print "WIN!"
    myOLED.display() # Actually show what we've printed
    
    # Play the winning sound
    timed_tone(1318, 150)  # E6
    sleep_ms(175)
    timed_tone(1567, 150)  # G6
    sleep_ms(175)
    timed_tone(2637, 150)  # E7
    sleep_ms(175)
    timed_tone(2093, 150)  # C7
    sleep_ms(175)
    timed_tone(2349, 150)  # D7
    sleep_ms(175)
    timed_tone(3135, 500)  # G7
    sleep_ms(500)

# Now let's actually run the game!
# Initialize the OLED display
myOLED.begin()  # Initialize the OLED display

generate_random_order()  # Generate a random order of words
show_start_sequence()  # Show the start sequence on the OLED display

print("Starting Program!")

# Variable for keeping track if the player has lost yet
playerHasLost = False 

for i in range(len(orderOfWords)):
    clear_all_oled()  # Clear the OLED display

    myOLED.set_cursor(0, 25)  # Move the cursor to the start of the middle(ish) row of the screen
    roundNumber = i + 1 # the array starts at 0, but the roundNumber will start counting from 1
    myOLED.print(str(roundNumber) + ": ")  # Print the current round number
    myOLED.print(words[orderOfWords[i]])  # Print the word from the random order list
    myOLED.display() # Actually show what we've printed

    startTime = ticks_ms()  # Get the current time in milliseconds
    
    # Loop until the button is pressed
    while buttonPin.value() == 1:  # Wait for the button to be pressed
        roundedTimeLeft = int((timeLimit - (ticks_ms() - startTime)) / 1000)  # Calculate the remaining time in seconds

        # Display the time left in the lower right corner of the OLED
        myOLED.set_cursor(110, 50)

        myOLED.print("  ")  # Clear the previous time display
        myOLED.set_cursor(110, 50)
        myOLED.print(str(roundedTimeLeft))
        myOLED.display() # Actually show what we've printed
        sleep_ms(15)

        # Check if the time limit has been exceeded
        if ticks_ms() - startTime > timeLimit:
            playerHasLost = True # Player has failed to meet the time limit so let's take note of that so we know to play the failure sequence
            break # Break Out of our while loop
        
        # If the button was pressed, we will play a short beep sound
        if buttonPin.value() == 0:
            timed_tone(272, 30)

    # If the player has lost, let's not show them the other words, and break out of our for loop
    if playerHasLost == True:
        game_over() # Show the game over message
        break # Break out of our "for" loop

    sleep_ms(500) # Wait for half a second to "debounce" the button press

# If we reach here and the whole for loop has completed without saying the player has lost,
# let's play their victory sequence
if playerHasLost == False:
    winner()  # Display the winning message

## What You Should See
The game will begin with a prompt telling you the category of words. Then it will run through a short countdown. When the first round starts, the word to be guessed will be displayed in the top left, and a countdown will be displayed in the bottom right of the LCD screen. Each time the button is pressed (before the timer expires) a new word will be displayed. If you win or lose, a short song will play and text will be displayed. Unplug and replug the battery pack or click the "RESET" buton on the RedBoard to restart the game.

## You've Completed Circuit 4C!

## You've Also Completed All of Project 4!

Continue to project 5 to learn about motors and build your own robot!

![Next - Circuit c](images/sik-demo-prj4-cc-next.png)